### Binary classification tutorial is implemented using FNN to detect network anomaly

Prepare Dataset

In [ ]:
import pandas as pd
import torch

sheet_id = "1VZoC4UF-I6rlljNwpqyNw2UNchA2LLH7ZN4eL4Om24A"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv"

UNSW = pd.read_csv(url)

In [ ]:
UNSW.columns

Index(['id', 'dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes',
       'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss',
       'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin',
       'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth',
       'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
       'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm',
       'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat', 'label'],
      dtype='object')

In [ ]:
for each in UNSW:
    if UNSW[each].dtype == "object":
       print(f"{each} : {UNSW[each].unique()}")

proto : ['udp' 'arp' 'tcp' 'igmp' 'ospf' 'sctp' 'gre' 'ggp' 'ip' 'ipnip' 'st2'
 'argus' 'chaos' 'egp' 'emcon' 'nvp' 'pup' 'xnet' 'mux' 'dcn' 'hmp' 'prm'
 'trunk-1' 'trunk-2' 'xns-idp' 'leaf-1' 'leaf-2' 'irtp' 'rdp' 'netblt'
 'mfe-nsp' 'merit-inp' '3pc' 'idpr' 'ddp' 'idpr-cmtp' 'tp++' 'ipv6' 'sdrp'
 'ipv6-frag' 'ipv6-route' 'idrp' 'mhrp' 'i-nlsp' 'rvd' 'mobile' 'narp'
 'skip' 'tlsp' 'ipv6-no' 'any' 'ipv6-opts' 'cftp' 'sat-expak' 'ippc'
 'kryptolan' 'sat-mon' 'cpnx' 'wsn' 'pvp' 'br-sat-mon' 'sun-nd' 'wb-mon'
 'vmtp' 'ttp' 'vines' 'nsfnet-igp' 'dgp' 'eigrp' 'tcf' 'sprite-rpc' 'larp'
 'mtp' 'ax.25' 'ipip' 'aes-sp3-d' 'micp' 'encap' 'pri-enc' 'gmtp' 'ifmp'
 'pnni' 'qnx' 'scps' 'cbt' 'bbn-rcc' 'igp' 'bna' 'swipe' 'visa' 'ipcv'
 'cphb' 'iso-tp4' 'wb-expak' 'sep' 'secure-vmtp' 'xtp' 'il' 'rsvp' 'unas'
 'fc' 'iso-ip' 'etherip' 'pim' 'aris' 'a/n' 'ipcomp' 'snp' 'compaq-peer'
 'ipx-n-ip' 'pgm' 'vrrp' 'l2tp' 'zero' 'ddx' 'iatp' 'stp' 'srp' 'uti' 'sm'
 'smp' 'isis' 'ptp' 'fire' 'crtp' 'crudp' 'scco

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded = encoder.fit_transform(UNSW[["proto","service","state"]])
feature_names = encoder.get_feature_names_out(["proto","service","state"])
encoded_df = pd.DataFrame(encoded, columns = feature_names)

In [ ]:
UNSW = UNSW.drop(["proto","service","state","id","attack_cat"], axis = 1)

In [ ]:
UNSW_DATASET = pd.concat([UNSW, encoded_df], axis = 1)

In [ ]:
train = UNSW_DATASET.drop("label", axis = 1)
test = UNSW_DATASET[["label"]]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train, test, test_size=0.2, shuffle = True)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test  = scaler.transform(x_test)


In [ ]:
x_train = torch.tensor(x_train, dtype=torch.float32)
x_test  = torch.tensor(x_test, dtype=torch.float32)

y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_test  = torch.tensor(y_test.values, dtype=torch.float32)

In [ ]:
pos_weight = (y_train == 0).sum() / (y_train == 1).sum()

In [ ]:
pos_weight

tensor(0.8198)

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

batch_size = 64

train_dataset = TensorDataset(x_train, y_train)
test_dataset  = TensorDataset(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
x,y = next(iter(train_loader))
print(x.shape, y.shape)

torch.Size([64, 190]) torch.Size([64, 1])


In [ ]:
from torch.nn import Linear, Sigmoid, ReLU, Module, Sequential, BCEWithLogitsLoss, Dropout
from torch.optim import RMSprop, Adam

Model

In [ ]:
class BinaryClassificationModel(Module):
      def __init__(self, input_shape):
         super().__init__()
         self.sequence = Sequential(
             Linear(input_shape, 60),
             ReLU(),
             Linear(60, 10),
             ReLU(),
             Dropout(0.2),
             Linear(10, 1)
         )
      def forward(self, x):
         return self.sequence(x)

In [ ]:
binary_classification_model = BinaryClassificationModel(input_shape = 190)
loss_function =  BCEWithLogitsLoss(pos_weight=pos_weight) # used in multi-label classification, binary classification etc
optimizer = Adam(params=binary_classification_model.parameters(), lr=0.001)

Training Function

In [ ]:
def training_func(model, criterion, optimizer, train_dataset):
    no_of_sample = len(train_loader.dataset)
    model.train()
    for batch,(x,y) in enumerate(train_dataset):

        prediction = model(x)

        loss = criterion(prediction, y)
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        if batch%100 == 0:
           print(f"train_loss : {loss.item():.4f} | complete : [{(batch+1) * x.shape[0]}/{no_of_sample}]")

Validation Function

In [ ]:
def test_func(model, criterion, test_dataset, threshold=0.5):
    no_of_sample = len(test_dataset.dataset)
    no_of_batch = len(test_dataset)
    model.eval()
    loss, accuracy = 0, 0
    with torch.no_grad():
      for x, y in test_dataset:

          prediction = model(x)

          loss += criterion(prediction, y).item()

          final_prediction = (torch.sigmoid(prediction) >= threshold)
          accuracy += (final_prediction == y).sum().item()

      loss /= no_of_batch
      accuracy /= no_of_sample

      print(f"test_loss : {loss:.4f} | accuracy : {accuracy*100:.4f}")
      return loss


Early Stopping -> prevents overfitting

In [ ]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = float('inf')

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss - self.min_delta:
            self.min_validation_loss = validation_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False


Model Training & Validation Cycle

In [ ]:
epoch = 8
early =  EarlyStopper()
for e in range(1, epoch+1):
   print(f"epoch : {e}")
   training_func(binary_classification_model, loss_function, optimizer, train_loader)
   loss = test_func(binary_classification_model, loss_function, test_loader)
   if early.early_stop(loss):
      break

epoch : 1
train_loss : 0.6362 | complete : [64/65865]
train_loss : 0.3278 | complete : [6464/65865]
train_loss : 0.2287 | complete : [12864/65865]
train_loss : 0.2019 | complete : [19264/65865]
train_loss : 0.1369 | complete : [25664/65865]
train_loss : 0.2117 | complete : [32064/65865]
train_loss : 0.1685 | complete : [38464/65865]
train_loss : 0.1625 | complete : [44864/65865]
train_loss : 0.1360 | complete : [51264/65865]
train_loss : 0.1093 | complete : [57664/65865]
train_loss : 0.1102 | complete : [64064/65865]
test_loss : 0.1417 | accuracy : 93.5082
epoch : 2
train_loss : 0.1722 | complete : [64/65865]
train_loss : 0.1049 | complete : [6464/65865]
train_loss : 0.1953 | complete : [12864/65865]
train_loss : 0.1117 | complete : [19264/65865]
train_loss : 0.1476 | complete : [25664/65865]
train_loss : 0.0792 | complete : [32064/65865]
train_loss : 0.1366 | complete : [38464/65865]
train_loss : 0.1257 | complete : [44864/65865]
train_loss : 0.1286 | complete : [51264/65865]
train_lo